In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
pd.set_option("display.max_columns", 100)
tr_set = pd.read_csv('data/training_set_features.csv', index_col='respondent_id')  # import training set of data
tr_set_labels = pd.read_csv('data/training_set_labels.csv', index_col='respondent_id')  #  import result data

np.testing.assert_array_equal(tr_set.index.values, tr_set_labels.index.values)  #  verify rows between features & labels match, nothing returned is good

tr_join = tr_set.join(tr_set_labels) #  Join testing data & results

In [ ]:
#  Formula for displaying % total NaN and data type

def nandtype():
	dt = tr_join.dtypes
	tr_cnames = tr_join.columns
	
	tot = tr_join.shape[0]  #  total row count for calculations
	ana = pd.DataFrame(columns=['Feature', 'Percent NaN', 'Data Type'])  #  creating data frame for easy visulation of data

	pn = tr_join.isna().sum()/tot*100  #  Percent of NaN values of total per Feature

	#using for loop to fill in data frame
	for feat in tr_cnames:
		ana = ana.append({'Feature':feat, 'Percent NaN':pn[feat], 'Data Type':dt[feat]}, ignore_index=True)

	ana.set_index('Feature', drop = True, inplace=True)
	return ana


In [ ]:
#  Basic data frame information
print(tr_join.shape)
dt = tr_join.dtypes  #  listing column data types and storing as a variable for future use
print(dt)
tr_cnames = tr_join.columns #  Assign column names to variable for potential future use
print(list(tr_cnames))  #  Column names are acceptable, I will alter some to make more sense for boolean data types

(26707, 37)
h1n1_concern                   float64
h1n1_knowledge                 float64
behavioral_antiviral_meds      float64
behavioral_avoidance           float64
behavioral_face_mask           float64
behavioral_wash_hands          float64
behavioral_large_gatherings    float64
behavioral_outside_home        float64
behavioral_touch_face          float64
doctor_recc_h1n1               float64
doctor_recc_seasonal           float64
chronic_med_condition          float64
child_under_6_months           float64
health_worker                  float64
health_insurance               float64
opinion_h1n1_vacc_effective    float64
opinion_h1n1_risk              float64
opinion_h1n1_sick_from_vacc    float64
opinion_seas_vacc_effective    float64
opinion_seas_risk              float64
opinion_seas_sick_from_vacc    float64
age_group                       object
education                       object
race                            object
sex                             object
income_povert

In [ ]:
#  Checking data integrity
print(tr_join.nunique())  #  values fall in expected ranges, boolean=2, categorical 3<x
print('hhs_geo_region: ', tr_join['hhs_geo_region'].unique())  # this feature is fine, no missing data
print('employment_industry: ', list(tr_join['employment_industry'].unique())) #  has missing data as nan
print('employment_occupation: ', list(tr_join['employment_occupation'].unique()))  #  has missing data as nan
print('Income Poverty: ', list(tr_join['income_poverty'].unique()))  #  has missing data as nan

#  Features with highest rate of unique values appear valid.  This information is randomized for privacy but still structured.
#  We won't know what employment industry or geographic location each value corresponds to but we will at least know if there 
#  is a correlation to the final outcome.

nandtype()

h1n1_concern                    4
h1n1_knowledge                  3
behavioral_antiviral_meds       2
behavioral_avoidance            2
behavioral_face_mask            2
behavioral_wash_hands           2
behavioral_large_gatherings     2
behavioral_outside_home         2
behavioral_touch_face           2
doctor_recc_h1n1                2
doctor_recc_seasonal            2
chronic_med_condition           2
child_under_6_months            2
health_worker                   2
health_insurance                2
opinion_h1n1_vacc_effective     5
opinion_h1n1_risk               5
opinion_h1n1_sick_from_vacc     5
opinion_seas_vacc_effective     5
opinion_seas_risk               5
opinion_seas_sick_from_vacc     5
age_group                       5
education                       4
race                            4
sex                             2
income_poverty                  3
marital_status                  2
rent_or_own                     2
employment_status               3
hhs_geo_region

,Percent NaN,Data Type
Feature,,
h1n1_concern,0.344479,float64
h1n1_knowledge,0.434343,float64
behavioral_antiviral_meds,0.265848,float64
behavioral_avoidance,0.778822,float64
behavioral_face_mask,0.071142,float64
behavioral_wash_hands,0.157262,float64
behavioral_large_gatherings,0.325757,float64
behavioral_outside_home,0.307036,float64
behavioral_touch_face,0.479275,float64


In [ ]:
#  There are numerous NaN entries we will have to look at each feature and drop or fill.  Anything under 10% will be row dropped
#  health_insurance will be filled in equating NaN to False, employment_industry & employment_occupation will be column dropped.
#  There are too many missing entries and no feasable way to fill in.  I will attempt to fill in income poverty based on other
#  Features and logical thinking.

In [ ]:
tr_join['employment_status'].unique()

array(['Not in Labor Force', 'Employed', 'Unemployed', nan], dtype=object)

In [ ]:
#  Fixing features that would be better served as bool type after some manipulation

tr_join.marital_status.replace({'Not Married':False , 'Married':True}, inplace=True)
tr_join['marital_status'] = tr_join['marital_status'].astype('bool')

tr_join['sex'].replace({'Male':False , 'Female':True }, inplace=True)
tr_join['sex'] = tr_join['sex'].astype('bool')

tr_join['rent_or_own'].replace({'Rent':False , 'Own':True }, inplace=True)
tr_join['rent_or_own'] = tr_join['rent_or_own'].astype('bool')

#  setting Feature names to be more intuitive
tr_join.rename(columns={'marital_status':'married', 'sex':'sex_m0f1', 'rent_or_own':'own_home'}, inplace = True)

print(tr_join.dtypes)

h1n1_concern                   float64
h1n1_knowledge                 float64
behavioral_antiviral_meds      float64
behavioral_avoidance           float64
behavioral_face_mask           float64
behavioral_wash_hands          float64
behavioral_large_gatherings    float64
behavioral_outside_home        float64
behavioral_touch_face          float64
doctor_recc_h1n1               float64
doctor_recc_seasonal           float64
chronic_med_condition          float64
child_under_6_months           float64
health_worker                  float64
health_insurance               float64
opinion_h1n1_vacc_effective    float64
opinion_h1n1_risk              float64
opinion_h1n1_sick_from_vacc    float64
opinion_seas_vacc_effective    float64
opinion_seas_risk              float64
opinion_seas_sick_from_vacc    float64
age_group                       object
education                       object
race                            object
sex_m0f1                          bool
income_poverty           

In [ ]:
#  Convert to approprate data types
#  Convert to type category
tr_join['h1n1_concern'] = tr_join['h1n1_concern'].astype('category')
tr_join['h1n1_knowledge'] = tr_join['h1n1_knowledge'].astype('category')
tr_join['opinion_h1n1_vacc_effective'] = tr_join['opinion_h1n1_vacc_effective'].astype('category')
tr_join['opinion_h1n1_risk'] = tr_join['opinion_h1n1_risk'].astype('category')
tr_join['opinion_h1n1_sick_from_vacc'] = tr_join['opinion_h1n1_sick_from_vacc'].astype('category')
tr_join['opinion_seas_vacc_effective'] = tr_join['opinion_seas_vacc_effective'].astype('category')
tr_join['opinion_seas_risk'] = tr_join['opinion_seas_risk'].astype('category')
tr_join['opinion_seas_sick_from_vacc'] = tr_join['opinion_seas_sick_from_vacc'].astype('category')
tr_join['household_adults'] = tr_join['household_adults'].astype('category')
tr_join['household_children'] = tr_join['household_children'].astype('category')
tr_join['age_group'] = tr_join['age_group'].astype('category')
tr_join['education'] = tr_join['education'].astype('category')
tr_join['race'] = tr_join['race'].astype('category')
tr_join['hhs_geo_region'] = tr_join['hhs_geo_region'].astype('category')
tr_join['census_msa'] = tr_join['census_msa'].astype('category')
tr_join['employment_status'] = tr_join['employment_status'].astype('category')
#  Due to remediation steps income_poverty will be converted at the end

#  Convert to boolean
tr_join['behavioral_antiviral_meds'] = tr_join['behavioral_antiviral_meds'].astype('bool')
tr_join['behavioral_avoidance'] = tr_join['behavioral_avoidance'].astype('bool')
tr_join['behavioral_face_mask'] = tr_join['behavioral_face_mask'].astype('bool')
tr_join['behavioral_wash_hands'] = tr_join['behavioral_wash_hands'].astype('bool')
tr_join['behavioral_large_gatherings'] = tr_join['behavioral_large_gatherings'].astype('bool')
tr_join['behavioral_outside_home'] = tr_join['behavioral_outside_home'].astype('bool')
tr_join['behavioral_touch_face'] = tr_join['behavioral_touch_face'].astype('bool')
tr_join['doctor_recc_h1n1'] = tr_join['doctor_recc_h1n1'].astype('bool')
tr_join['doctor_recc_seasonal'] = tr_join['doctor_recc_seasonal'].astype('bool')
tr_join['chronic_med_condition'] = tr_join['chronic_med_condition'].astype('bool')
tr_join['child_under_6_months'] = tr_join['child_under_6_months'].astype('bool')
tr_join['health_worker'] = tr_join['health_worker'].astype('bool')
tr_join['health_insurance'] = tr_join['health_insurance'].astype('bool')
tr_join['h1n1_vaccine'] = tr_join['h1n1_vaccine'].astype('bool')
tr_join['seasonal_vaccine'] = tr_join['seasonal_vaccine'].astype('bool')

print(tr_join.dtypes)

h1n1_concern                   category
h1n1_knowledge                 category
behavioral_antiviral_meds          bool
behavioral_avoidance               bool
behavioral_face_mask               bool
behavioral_wash_hands              bool
behavioral_large_gatherings        bool
behavioral_outside_home            bool
behavioral_touch_face              bool
doctor_recc_h1n1                   bool
doctor_recc_seasonal               bool
chronic_med_condition              bool
child_under_6_months               bool
health_worker                      bool
health_insurance                   bool
opinion_h1n1_vacc_effective    category
opinion_h1n1_risk              category
opinion_h1n1_sick_from_vacc    category
opinion_seas_vacc_effective    category
opinion_seas_risk              category
opinion_seas_sick_from_vacc    category
age_group                      category
education                      category
race                           category
sex_m0f1                           bool


In [ ]:
print(tr_join.shape)
#  dropping employment_industry & employment_occupation columns
tr_join.drop(columns=['employment_industry', 'employment_occupation'], inplace=True)

dt = tr_join.dtypes
tr_cat = tr_join.nunique()[tr_join.nunique() > 2].keys()  #  Features of category data type, storing for future use
tr_bol = tr_join.nunique()[tr_join.nunique() <= 2].keys()  #  Features of boolean data type, storing for future use
tr_cnames = tr_join.columns

#  For Boolean features we will set any values of NaN to 0 which is logical in these binary choices
for feat in tr_bol:
    tr_join[feat].fillna(0)
    
nandtype()

(26707, 37)


,Percent NaN,Data Type
Feature,,
h1n1_concern,0.344479,category
h1n1_knowledge,0.434343,category
behavioral_antiviral_meds,0.000000,bool
behavioral_avoidance,0.000000,bool
behavioral_face_mask,0.000000,bool
behavioral_wash_hands,0.000000,bool
behavioral_large_gatherings,0.000000,bool
behavioral_outside_home,0.000000,bool
behavioral_touch_face,0.000000,bool


In [ ]:
#  For category data types the rows with missing data will be dropped since it is such a small percent of the total per feature.
tr_cat = list(tr_cat)
tr_cat.remove('income_poverty')
print(tr_cat)  #  List of features of data type 'category' minus 'income poverty' which will be dealth with special

tr_join = tr_join.dropna(subset = tr_cat)  #  dropping all Na value rows for features of 'category' data type
print(tr_join.shape)

nandtype()

['h1n1_concern', 'h1n1_knowledge', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk', 'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective', 'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group', 'education', 'race', 'employment_status', 'hhs_geo_region', 'census_msa', 'household_adults', 'household_children']
(24761, 35)


,Percent NaN,Data Type
Feature,,
h1n1_concern,0.000000,category
h1n1_knowledge,0.000000,category
behavioral_antiviral_meds,0.000000,bool
behavioral_avoidance,0.000000,bool
behavioral_face_mask,0.000000,bool
behavioral_wash_hands,0.000000,bool
behavioral_large_gatherings,0.000000,bool
behavioral_outside_home,0.000000,bool
behavioral_touch_face,0.000000,bool


In [ ]:
print('Total NaN in income_poverty: ', tr_join['income_poverty'].isna().sum())
print('Total NaN in income_poverty with health insurance: ', tr_join['income_poverty'][tr_join['health_insurance'] == True].isna().sum())
print('Total NaN in income_poverty who own a home: ', tr_join['income_poverty'][tr_join['own_home'] == True].isna().sum())
print('Total NaN in income_poverty who are Employed: ', tr_join['income_poverty'][tr_join['employment_status'] == 'Employed'].isna().sum())

print('Total NaN in income_poverty without health insurance: ', tr_join['income_poverty'][tr_join['health_insurance'] == False].isna().sum())
print('Total NaN in income_poverty who own rent: ', tr_join['income_poverty'][tr_join['own_home'] == False].isna().sum())
print('Total NaN in income_poverty who arent Employed: ', tr_join['income_poverty'][tr_join['employment_status'] == ('Not in Labor Force' or 'Unemployed')].isna().sum())

above = tr_join['health_insurance' and 'own_home' and 'employment_status']
            

Total NaN in income_poverty:  2845
Total NaN in income_poverty with health insurance:  2646
Total NaN in income_poverty who own a home:  2252
Total NaN in income_poverty who are Employed:  1086
Total NaN in income_poverty without health insurance:  199
Total NaN in income_poverty who own rent:  593
Total NaN in income_poverty who arent Employed:  1581


  There are 3 valid responses for income_poverty, 'Below Poverty', '<= $75,000, Above Poverty' & '> $75,000'
  Thinking about the data critically, there aren't any other features that would reasonably imply which category an item would fall under.  Therefore I have decided to treat the NaN's as their own 4th answer category.  I'll convert them to 'Not Answered'  

In [ ]:
tr_join.income_poverty.fillna('Not Answered', inplace=True)
tr_join['income_poverty'] = tr_join['income_poverty'].astype('category')
print('Income Poverty: ', list(tr_join['income_poverty'].unique()))
nandtype()

,Percent NaN,Data Type
Feature,,
h1n1_concern,0.0,category
h1n1_knowledge,0.0,category
behavioral_antiviral_meds,0.0,bool
behavioral_avoidance,0.0,bool
behavioral_face_mask,0.0,bool
behavioral_wash_hands,0.0,bool
behavioral_large_gatherings,0.0,bool
behavioral_outside_home,0.0,bool
behavioral_touch_face,0.0,bool


In [ ]:
#  Output cleaned data
tr_join.to_csv('tr_clean.csv', index=False)